In [4]:
!pip install rouge

In [5]:
import pandas as pd
from transformers import pipeline
from rouge import Rouge

In [6]:
#train data
train_data = pd.read_csv('/content/train.csv')

In [7]:
#test data
test_data = pd.read_csv('/content/test.csv')

In [8]:
#validation data
val_data = pd.read_csv('/content/validation.csv')

In [9]:
train_data

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [10]:
val_data = val_data.sample(n=10, random_state=42)

In [11]:
summarizer = pipeline("summarization", model="facebook/bart-base")
summarizer.model.save_pretrained("/content/bart_no_training")
summarizer.tokenizer.save_pretrained("/content/bart_no_training")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('/content/bart_no_training/tokenizer_config.json',
 '/content/bart_no_training/special_tokens_map.json',
 '/content/bart_no_training/vocab.json',
 '/content/bart_no_training/merges.txt',
 '/content/bart_no_training/added_tokens.json',
 '/content/bart_no_training/tokenizer.json')

In [12]:
# Generate summaries for each dialogue in the test data
summaries = []
for text in val_data['dialogue']:
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    summaries.append(summary)

Your max_length is set to 150, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 150, but your input_length is only 149. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)
Your max_length is set to 150, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)
Your max_length is set to 150, but your input_length is only 138. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


In [13]:
# Add the generated summaries to the test data DataFrame
val_data['generated_summary'] = summaries

In [14]:
val_data = val_data[["dialogue", "summary", "generated_summary"]]

In [15]:
val_data

,dialogue,summary,generated_summary
361,"#Person1#: Trina, will you marry me?\n#Person2...","Trina accepts Jared's proposal. Then, Jared is...","#Person1#: Trina, will you marry me? What's yo..."
73,#Person1#: There have been too many unplanned ...,#Person1# proposes to build maintenance proced...,#Person1#: There have been too many unplanned ...
374,"#Person1#: Hello, is this house keeper?\n#Pers...",Terry Chen in Room 117 calls the housekeeper f...,"#Person1#: Hello, is this house keeper? I'm Te..."
155,#Person1#: I want to get on the bus already.\n...,#Person1# and #Person2# have been waiting for ...,#Person1#: I want to get on the bus already. I...
104,#Person1#: we really were lucky. We got the la...,#Person1# and #Person2# are discussing what to...,#Person1#: we really were lucky. We got the la...
394,#Person1#: I'm planning to go to Canada on vac...,#Person1# tells #Person2# about #Person1#'s va...,#Person1#: I'm planning to go to Canada on vac...
377,"#Person1#: That is the most boring, typical gi...",#Person1# dislikes #Person2#'s idea of getting...,"#Person1#: That is the most boring, typical gi..."
124,#Person1#: Dental clinic. This is Mr. Adams.\n...,#Person2# calls #Person1# to make an appointme...,#Person1#: Dental clinic. This is Mr. Adams. H...
68,#Person1#: We can offer you a 5 % discount.\n#...,#Person1# offers a discount but #Person2# is n...,#Person1#: We can offer you a 5 % discount. Ho...
450,"#Person1#: David, we have been doing business ...","After three years of cooperation, #Person1# ap...","#Person1#: David, we have been doing business ..."


In [16]:
# Initialize Rouge
rouge = Rouge()

# Calculate Rouge scores
rouge_scores = []
for index, row in val_data.iterrows():
    rouge_score = rouge.get_scores(row['generated_summary'], row['summary'])
    rouge_scores.append(rouge_score[0])  # Assuming you have a single reference summary

# Create a DataFrame with ROUGE scores
rouge_df = pd.DataFrame(rouge_scores)


In [17]:
def round_dict_values(d):
    return {key: round(value, 2) if isinstance(value, (int, float)) else value for key, value in d.items()}

# Apply the rounding function to each element in the DataFrame
rouge_df = rouge_df.applymap(round_dict_values)

In [18]:
rouge_df

,rouge-1,rouge-2,rouge-l
0,"{'r': 0.08, 'p': 0.07, 'f': 0.07}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.08, 'p': 0.07, 'f': 0.07}"
1,"{'r': 0.64, 'p': 0.09, 'f': 0.15}","{'r': 0.27, 'p': 0.03, 'f': 0.05}","{'r': 0.64, 'p': 0.09, 'f': 0.15}"
2,"{'r': 0.36, 'p': 0.08, 'f': 0.12}","{'r': 0.08, 'p': 0.01, 'f': 0.02}","{'r': 0.36, 'p': 0.08, 'f': 0.12}"
3,"{'r': 0.32, 'p': 0.12, 'f': 0.17}","{'r': 0.15, 'p': 0.04, 'f': 0.07}","{'r': 0.32, 'p': 0.12, 'f': 0.17}"
4,"{'r': 0.25, 'p': 0.08, 'f': 0.12}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.15, 'p': 0.05, 'f': 0.07}"
5,"{'r': 0.44, 'p': 0.06, 'f': 0.1}","{'r': 0.25, 'p': 0.02, 'f': 0.04}","{'r': 0.33, 'p': 0.04, 'f': 0.08}"
6,"{'r': 0.3, 'p': 0.09, 'f': 0.14}","{'r': 0.05, 'p': 0.01, 'f': 0.02}","{'r': 0.3, 'p': 0.09, 'f': 0.14}"
7,"{'r': 0.7, 'p': 0.1, 'f': 0.17}","{'r': 0.56, 'p': 0.05, 'f': 0.1}","{'r': 0.7, 'p': 0.1, 'f': 0.17}"
8,"{'r': 0.2, 'p': 0.04, 'f': 0.07}","{'r': 0.06, 'p': 0.01, 'f': 0.02}","{'r': 0.2, 'p': 0.04, 'f': 0.07}"
9,"{'r': 0.35, 'p': 0.17, 'f': 0.23}","{'r': 0.2, 'p': 0.08, 'f': 0.11}","{'r': 0.29, 'p': 0.14, 'f': 0.19}"


In [19]:
import re
def clean_text(text):
    # Remove HTML-like tags and special instructions
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"\[/?INST\]", "", text)
    text = re.sub(r"Agent \d+:", "Agent says:", text)
    text = re.sub(r"Customer:", "Customer says:", text)
    return text

In [20]:
sample_text = "There have been too many unplanned shutdowns over the past few months. We'll have to make a few changes to the way we operate. We need to reduce our downtime. #Person2#: I agree, but the maintenance team is fully stretched dealing with problems. We don't have time to carry out any preventive maintenance. #Person1#: We need to establish routine maintenance procedures. It costs us too much in lost production if we wait until something breaks down before we fix it."

In [21]:
text = clean_text(sample_text)

In [22]:
text

"There have been too many unplanned shutdowns over the past few months. We'll have to make a few changes to the way we operate. We need to reduce our downtime. #Person2#: I agree, but the maintenance team is fully stretched dealing with problems. We don't have time to carry out any preventive maintenance. #Person1#: We need to establish routine maintenance procedures. It costs us too much in lost production if we wait until something breaks down before we fix it."

In [23]:
result = print(summarizer(text, max_length=130, min_length=30, do_sample=False))

Your max_length is set to 130, but your input_length is only 102. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "There have been too many unplanned shutdowns over the past few months. We'll have to make a few changes to the way we operate. We need to reduce our downtime. #Person2#: I agree, but the maintenance team is fully stretched dealing with problems. We don't have time to carry out any preventive maintenance. # Person1#: We need a system in place that allows us to perform routine maintenance procedures. It costs us too much in lost production if we wait until something breaks down before we fix it."}]
